<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/kaggle2025stanford_RNA_3d_folding_deepseek_gpt4o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ========== 環境設置 ==========
# 若在 Colab 運行需先執行以下命令：
!pip install torch==2.3.0+cu121 torchvision==0.18.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121
!pip install tensorboard

# ========== 導入函式庫 ==========
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import GradScaler, autocast
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path
import numpy as np

# ========== 超參數設定類別 ==========
class 訓練設定:
    # 硬體設定
    運算裝置 = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 資料參數
    批次大小 = 128
    驗證集比例 = 0.2  # 從訓練集分割驗證集的比例

    # 優化參數
    初始學習率 = 1e-3
    權重衰減 = 1e-4
    訓練週期數 = 50
    早停耐心值 = 5  # 驗證集精度未提升時等待的週期數

    # 模型儲存
    模型儲存路徑 = './最佳模型.pth'

    # TensorBoard紀錄
    日誌目錄 = './訓練紀錄/實驗1'

設定 = 訓練設定()
Path(設定.日誌目錄).mkdir(parents=True, exist_ok=True)  # 自動建立日誌目錄

# ========== 資料增強與標準化 ==========
CIFAR10_均值 = (0.4914, 0.4822, 0.4465)
CIFAR10_標準差 = (0.2023, 0.1994, 0.2010)

# 訓練資料轉換（含數據增強）
訓練資料轉換 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),       # 隨機裁剪
    transforms.RandomHorizontalFlip(),          # 水平翻轉
    transforms.RandomRotation(15),              # 隨機旋轉
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # 顏色抖動
    transforms.ToTensor(),                      # 轉換為張量
    transforms.Normalize(CIFAR10_均值, CIFAR10_標準差)  # 標準化
])

# 測試資料轉換（無數據增強）
測試資料轉換 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_均值, CIFAR10_標準差)
])

# ========== 資料集載入與分割 ==========
# 下載並載入CIFAR10資料集
訓練資料集 = torchvision.datasets.CIFAR10(
    root='./資料', train=True, download=True, transform=訓練資料轉換
)
測試資料集 = torchvision.datasets.CIFAR10(
    root='./資料', train=False, download=True, transform=測試資料轉換
)

# 分割訓練集與驗證集
訓練集數量 = int((1 - 設定.驗證集比例) * len(訓練資料集))
驗證集數量 = len(訓練資料集) - 訓練集數量
訓練子集, 驗證子集 = random_split(訓練資料集, [訓練集數量, 驗證集數量])

# 建立資料載入器
訓練集載入器 = DataLoader(
    訓練子集, batch_size=設定.批次大小, shuffle=True,
    num_workers=2, pin_memory=True, drop_last=True
)
驗證集載入器 = DataLoader(
    驗證子集, batch_size=設定.批次大小, shuffle=False,
    num_workers=2, pin_memory=True
)
測試集載入器 = DataLoader(
    測試資料集, batch_size=設定.批次大小, shuffle=False,
    num_workers=2, pin_memory=True
)

# ========== 卷積神經網路模型定義 ==========
class 影像分類模型(nn.Module):
    def __init__(self):
        super().__init__()
        # 特徵提取層
        self.特徵層 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),  # 輸入通道3，輸出通道64
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 下採樣至16x16

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 下採樣至8x8
        )

        # 分類層
        self.分類層 = nn.Sequential(
            nn.Linear(128*8*8, 512),
            nn.Dropout(0.5),  # 防止過擬合
            nn.Linear(512, 10) # 輸出10個類別
        )

    def forward(self, 輸入張量):
        特徵圖 = self.特徵層(輸入張量)
        展平特徵 = 特徵圖.view(特徵圖.size(0), -1)  # 展平為一維向量
        return self.分類層(展平特徵)

# 初始化模型並移至運算裝置
模型 = 影像分類模型().to(設定.運算裝置)
損失函數 = nn.CrossEntropyLoss()
優化器 = AdamW(模型.parameters(), lr=設定.初始學習率, weight_decay=設定.權重衰減)
學習率調度器 = CosineAnnealingLR(優化器, T_max=設定.訓練週期數)  # 餘弦退火調度
混合精度縮放器 = GradScaler()  # 自動混合精度訓練
日誌寫入器 = SummaryWriter(設定.日誌目錄)  # TensorBoard紀錄

# ========== 超參數紀錄 ==========
超參數組 = {
    '批次大小': 設定.批次大小,
    '初始學習率': 設定.初始學習率,
    '權重衰減': 設定.權重衰減,
    '優化器': type(優化器).__name__,
    '模型類型': type(模型).__name__,
}

# ========== 訓練迴圈 ==========
最佳驗證準確率 = 0.0
早停計數器 = 0

print(f"開始訓練，使用裝置：{設定.運算裝置}")
for 週期 in range(設定.訓練週期數):
    # 訓練階段
    模型.train()
    訓練損失累計 = 0.0
    訓練正確數 = 0

    for 批次輸入, 批次標籤 in 訓練集載入器:
        批次輸入 = 批次輸入.to(設定.運算裝置, non_blocking=True)
        批次標籤 = 批次標籤.to(設定.運算裝置, non_blocking=True)

        優化器.zero_grad()

        # 混合精度訓練上下文
        with autocast():
            模型輸出 = 模型(批次輸入)
            損失值 = 損失函數(模型輸出, 批次標籤)

        # 梯度縮放與反向傳播
        混合精度縮放器.scale(損失值).backward()
        混合精度縮放器.step(優化器)
        混合精度縮放器.update()

        # 累計訓練指標
        訓練損失累計 += 損失值.item() * 批次輸入.size(0)
        _, 預測結果 = torch.max(模型輸出, 1)
        訓練正確數 += (預測結果 == 批次標籤).sum().item()

    # 驗證階段
    模型.eval()
    驗證損失累計 = 0.0
    驗證正確數 = 0

    with torch.no_grad():
        for 批次輸入, 批次標籤 in 驗證集載入器:
            批次輸入 = 批次輸入.to(設定.運算裝置, non_blocking=True)
            批次標籤 = 批次標籤.to(設定.運算裝置, non_blocking=True)

            模型輸出 = 模型(批次輸入)
            損失值 = 損失函數(模型輸出, 批次標籤)

            驗證損失累計 += 損失值.item() * 批次輸入.size(0)
            _, 預測結果 = torch.max(模型輸出, 1)
            驗證正確數 += (預測結果 == 批次標籤).sum().item()

    # 計算平均指標
    平均訓練損失 = 訓練損失累計 / len(訓練子集)
    訓練準確率 = 訓練正確數 / len(訓練子集)
    平均驗證損失 = 驗證損失累計 / len(驗證子集)
    驗證準確率 = 驗證正確數 / len(驗證子集)

    # 更新學習率
    學習率調度器.step()

    # 寫入TensorBoard日誌
    日誌寫入器.add_scalar('損失/訓練集', 平均訓練損失, 週期)
    日誌寫入器.add_scalar('準確率/訓練集', 訓練準確率, 週期)
    日誌寫入器.add_scalar('損失/驗證集', 平均驗證損失, 週期)
    日誌寫入器.add_scalar('準確率/驗證集', 驗證準確率, 週期)

    # 紀錄權重與梯度直方圖
    for 參數名稱, 參數值 in 模型.named_parameters():
        日誌寫入器.add_histogram(f'權重/{參數名稱}', 參數值, 週期)
        日誌寫入器.add_histogram(f'梯度/{參數名稱}', 參數值.grad, 週期)

    # 早停機制與模型儲存
    if 驗證準確率 > 最佳驗證準確率:
        最佳驗證準確率 = 驗證準確率
        torch.save(模型.state_dict(), 設定.模型儲存路徑)
        早停計數器 = 0
    else:
        早停計數器 += 1
        if 早停計數器 >= 設定.早停耐心值:
            print(f'▄︻┻═┳一 在週期 {週期} 觸發早停機制')
            break

    # 輸出訓練進度
    print(f'週期 [{週期+1}/{設定.訓練週期數}] | '
          f'訓練損失：{平均訓練損失:.4f} 準確率：{訓練準確率:.4f} | '
          f'驗證損失：{平均驗證損失:.4f} 準確率：{驗證準確率:.4f}')

# ========== 最終測試評估 ==========
模型.load_state_dict(torch.load(設定.模型儲存路徑))
模型.eval()
測試正確數 = 0
with torch.no_grad():
    for 批次輸入, 批次標籤 in 測試集載入器:
        批次輸入 = 批次輸入.to(設定.運算裝置, non_blocking=True)
        批次標籤 = 批次標籤.to(設定.運算裝置, non_blocking=True)

        模型輸出 = 模型(批次輸入)
        _, 預測結果 = torch.max(模型輸出, 1)
        測試正確數 += (預測結果 == 批次標籤).sum().item()

測試準確率 = 測試正確數 / len(測試資料集)
print(f'✧✧✧ 最終測試準確率：{測試準確率:.4f} ✧✧✧')

# 紀錄超參數組合與測試結果
日誌寫入器.add_hparams(
    {參數名稱: str(參數值) for 參數名稱, 參數值 in 超參數組.items()},
    {'超參數組/測試準確率': 測試準確率}
)
日誌寫入器.close()

# 使用提示
print(f"\nTensorBoard 查看指令：tensorboard --logdir={設定.日誌目錄}")
print("可調整 訓練設定 類別中的參數進行超參數調校")

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

100%|██████████| 170498071/170498071 [00:03<00:00, 47450112.55it/s]


Extracting ./資料/cifar-10-python.tar.gz to ./資料
Files already downloaded and verified
開始訓練，使用裝置：cuda
週期 [1/50] | 訓練損失：2.8837 準確率：0.3198 | 驗證損失：1.6973 準確率：0.4093
週期 [2/50] | 訓練損失：1.5460 準確率：0.4375 | 驗證損失：1.4560 準確率：0.4714
週期 [3/50] | 訓練損失：1.4235 準確率：0.4871 | 驗證損失：1.3580 準確率：0.5017
週期 [4/50] | 訓練損失：1.3391 準確率：0.5168 | 驗證損失：1.2832 準確率：0.5364
週期 [5/50] | 訓練損失：1.2688 準確率：0.5447 | 驗證損失：1.2263 準確率：0.5655
週期 [6/50] | 訓練損失：1.2219 準確率：0.5603 | 驗證損失：1.1833 準確率：0.5758
週期 [7/50] | 訓練損失：1.1798 準確率：0.5803 | 驗證損失：1.1302 準確率：0.5988
週期 [8/50] | 訓練損失：1.1390 準確率：0.5941 | 驗證損失：1.1300 準確率：0.6013
週期 [9/50] | 訓練損失：1.1159 準確率：0.6055 | 驗證損失：1.0859 準確率：0.6180
週期 [10/50] | 訓練損失：1.0801 準確率：0.6149 | 驗證損失：1.1165 準確率：0.6149
週期 [11/50] | 訓練損失：1.0551 準確率：0.6259 | 驗證損失：1.0595 準確率：0.6308
週期 [12/50] | 訓練損失：1.0287 準確率：0.6351 | 驗證損失：1.0217 準確率：0.6425
週期 [13/50] | 訓練損失：1.0113 準確率：0.6430 | 驗證損失：1.0021 準確率：0.6484
週期 [14/50] | 訓練損失：0.9833 準確率：0.6536 | 驗證損失：0.9854 準確率：0.6596
週期 [15/50] | 訓練損失：0.9731 準確率：0.6574 | 驗證損失：0.9343 準確率：0